In [1]:
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import torchvision
from torchvision.models.detection import MaskRCNN

In [2]:
# Load the pre-trained Mask R-CNN model
model = torch.load("./finetuned_rcnn.pt")
model.eval()
print()

# RGB with HHA

In [3]:
ret = []
for i in tqdm(range(1449)):
    rgb = np.load("../../cv_corpus/nyudv2/rgb/" + str(i) + ".npy")/255
    hha = np.load("../../cv_corpus/nyudv2/hha/" + str(i) + ".npy")/255
    rgb = rgb.reshape(3,480,640).tolist()
    hha = hha.reshape(3,480,640).tolist()
    feature_rgb = model.backbone(torch.tensor(rgb))["pool"][0].reshape(256*8*10).tolist()
    feature_hha = model.backbone(torch.tensor(hha))["pool"][0].reshape(256*8*10).tolist()
    ret.append(feature_rgb + feature_hha)

100%|███████████████████████████████████████| 1449/1449 [19:06<00:00,  1.26it/s]


In [4]:
img = [i for i in range(1449)]

In [5]:
df_hha = pd.DataFrame(list(zip(img, ret)), columns =['Img_id', 'Embedding'])

In [7]:
df_hha.to_csv("./outs/RGBHHA_emb.csv")

# RGB with 3*D

In [10]:
ret = []
for i in tqdm(range(1449)):
    rgb = np.load("../../cv_corpus/nyudv2/rgb/" + str(i) + ".npy")/255
    dep = np.load("../../cv_corpus/nyudv2/depth/" + str(i) + ".npy")/255
    rgb = rgb.reshape(3,480,640).tolist()
    dep = dep.reshape(1,480,640).tolist()*3
    feature_rgb = model.backbone(torch.tensor(rgb))["pool"][0].reshape(256*8*10).tolist()
    feature_dep = model.backbone(torch.tensor(dep))["pool"][0].reshape(256*8*10).tolist()
    ret.append(feature_rgb + feature_dep)

100%|███████████████████████████████████████| 1449/1449 [19:12<00:00,  1.26it/s]


In [11]:
df_dep = pd.DataFrame(list(zip(img, ret)), columns =['Img_id', 'Embedding'])

In [12]:
df_dep.to_csv("./outs/RGBD_emb.csv")